In [1]:
import pandas as pd 
import requests
import os
import json
from pprint import pprint

In [2]:
n = 300
page = 7

In [3]:
responde = requests.get(f"https://marketplace.ifood.com.br/v2/merchants?latitude=-5.847599&longitude=-35.241176&page={page}&channel=IFOOD&size={n}&features=&categories=&payment_types=&delivery_fee_from=0&delivery_fee_to=25&delivery_time_from=0&delivery_time_to=240")

In [ ]:
responde.text

In [5]:
array = []

for i in range(0, 8):
  r = responde = requests.get(f"https://marketplace.ifood.com.br/v2/merchants?latitude=-5.847599&longitude=-35.241176&page={i}&channel=IFOOD&size={n}&features=&categories=&payment_types=&delivery_fee_from=0&delivery_fee_to=25&delivery_time_from=0&delivery_time_to=240")
  j = json.loads(r.text)
  array += j['merchants']

In [6]:
len(array)

2200

In [8]:
lojas = array

In [9]:
lojas[1]

{'available': True,
 'contextSetup': {'catalogGroup': 'ffca0022-eb43-4205-9a1b-73a72f8e3f95',
  'context': 'DEFAULT',
  'regionGroup': 'e1dbd9d8-45d6-4b33-aafc-417b8d69b06d'},
 'currency': 'BRL',
 'deliveryFee': {'originalValue': 8.9, 'type': 'FIXED', 'value': 8.9},
 'deliveryMethods': [{'deliveredBy': 'MERCHANT',
   'id': 'DEFAULT',
   'maxTime': 48,
   'minTime': 38,
   'mode': 'DELIVERY',
   'originalValue': 8.9,
   'priority': 1,
   'schedule': {'now': True, 'shifts': [], 'timeSlots': []},
   'subtitle': 'O entregador leva até você agora',
   'title': 'Padrão',
   'type': 'FIXED',
   'value': 8.9},
  {'id': 'TAKEOUT',
   'maxTime': 30,
   'minTime': 20,
   'mode': 'TAKEOUT',
   'priority': 4,
   'schedule': {'now': True, 'shifts': [], 'timeSlots': []},
   'subtitle': 'Você retira seu pedido no local',
   'title': 'Retirada',
   'type': 'FIXED',
   'value': 0.0},
  {'id': 'PICKUP_AREA',
   'maxTime': 30,
   'minTime': 20,
   'mode': 'TAKEOUT',
   'priority': 5,
   'schedule': {'now'

In [10]:
def get_data(json) -> list:
  try:
    id_ = json['id']
  except:
    id = ''

  try:
    nome = json['name'].strip()
  except:
    nome = ''

  try:
    resources = json['resources'][0]['fileName'].strip()
  except:
    resources = ''

  try:
    local = json['slug'].split("/")[0].strip()
  except:
    local = ''

  try:
    categoria = json['mainCategory']['name'].strip()
  except:
    categoria = ''

  try:
    taxa = json['deliveryFee']['value']
  except:
    taxa = ''

  try:
    minimo = json['minimumOrderValue']
  except:
    minimo = ''

  variaveis = locals()
  variaveis.pop("json")
  return list(variaveis.values())

In [11]:
get_data(lojas[5])

['0170dcfd-5aab-4bd5-b3af-07d7ed2b8ff1',
 'Churrascaria e Pizzaria Arnaldo Oliveira',
 '201710261058__CHURR_RANCO.png',
 'natal-rn',
 'Carnes',
 7.0,
 18.0]

In [12]:
res = list()

for json in lojas:
  res.append(get_data(json))

In [13]:
restaurantes = pd.DataFrame(res, columns = ['id', 'nome', 'logo', 'local', 'categoria', 'taxa', 'minimo'])
restaurantes

,id,nome,logo,local,categoria,taxa,minimo
0,6cc7fa43-032b-47be-954e-83d32a151acc,Gela Guela Sorveteria e Conveniência,6cc7fa43-032b-47be-954e-83d32a151acc/202005261...,natal-rn,Açaí,9.00,0.0
1,4b90132a-1614-431c-a38f-976b821ab176,Habib's - Salgado Filho,4b90132a-1614-431c-a38f-976b821ab176/202103031...,natal-rn,Lanches,8.90,10.0
2,d4a3984f-2b73-4f46-99df-1d6bc79ff293,China in Box - Tirol,d4a3984f-2b73-4f46-99df-1d6bc79ff293/202101041...,natal-rn,Chinesa,10.50,15.0
3,e0184c17-d62c-455c-93d8-c23ef40f2e3f,Click Pão Padaria e Restaurante,e0184c17-d62c-455c-93d8-c23ef40f2e3f/202001211...,parnamirim-rn,Padaria,5.00,16.0
4,1e9d77d2-f158-4a42-9eba-8b3cac08e6fd,Leve Culinária Brasileira & Chinesa,1e9d77d2-f158-4a42-9eba-8b3cac08e6fd/202003311...,natal-rn,Brasileira,4.00,15.0
...,...,...,...,...,...,...,...
2195,9aef61e4-64d9-4c6f-b32e-c293234dbd0c,Chicken Premium,9aef61e4-64d9-4c6f-b32e-c293234dbd0c/202102102...,natal-rn,Lanches,12.99,10.0
2196,1715b188-a013-4286-808c-3e5d166d4609,Lar do Cuscuz,1715b188-a013-4286-808c-3e5d166d4609/202103061...,natal-rn,Brasileira,7.49,20.0
2197,aa8fc4b7-be63-462b-8c35-994dc85287e7,Açai do Sué,aa8fc4b7-be63-462b-8c35-994dc85287e7/202102271...,natal-rn,Açaí,3.49,10.0
2198,f58184ea-46c1-4888-bfa1-f8ea67018a3a,Restaurante Mamacita,f58184ea-46c1-4888-bfa1-f8ea67018a3a/202103181...,natal-rn,Marmita,5.00,20.0


In [14]:
url = 'https://static-images.ifood.com.br/image/upload/t_thumbnail/logosgde/'

In [16]:
restaurantes['logo'] = restaurantes['logo'].apply(lambda x : url+x)

In [18]:
restaurantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         2200 non-null   object 
 1   nome       2200 non-null   object 
 2   logo       2200 non-null   object 
 3   local      2200 non-null   object 
 4   categoria  2200 non-null   object 
 5   taxa       2200 non-null   float64
 6   minimo     2200 non-null   float64
dtypes: float64(2), object(5)
memory usage: 120.4+ KB


In [25]:
restaurantes.iloc[10].logo

'https://static-images.ifood.com.br/image/upload/t_thumbnail/logosgde/29f0da49-a9ac-4cd1-b72b-5ef24ee8ee12/202105171502_5GIu_i.png'

In [28]:
!mkdir logos

In [44]:
def download_logos(id_: str, logo: str) -> None:
  logos = 'logos'

  posfixo = logo.split('.')[-1]
  file_name = f"{id_}.{posfixo}"

  file_path = os.path.join(os.getcwd(), logos, file_name)

  try:
   response = requests.get(logo)
  except:
    pass
  else:
   with open(file_path, 'wb') as f:
    f.write(response.content)


In [46]:
download_logos('id_qualquer', restaurantes.iloc[1].logo)

In [49]:
for _,row in restaurantes.iterrows():
  download_logos(row.id, row.logo)

In [50]:
restaurantes.to_csv('restaurantes_natal.csv', index=False)